In [1]:
import pandas as pd
import os
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import hmean
from pingouin import wilcoxon

sns.set_theme(style="whitegrid")

In [20]:
def compute_statistics(col_name, table_col, condition):
  for data in baseline_df['data'].unique():
    print(data)
    baseline_acc = baseline_df[baseline_df['data'] == data][col_name]
    for constraint in df_geg2['constraint'].unique():
      geg_acc = df_geg2[(df_geg2['data'] == data) & (df_geg2['constraint'] == constraint)][col_name]
      stat = wilcoxon(baseline_acc.abs(), geg_acc.abs(), alternative=condition)
      if stat['p-val'].values[0] <= 0.05:
        table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col].apply(lambda x: '\isBetter '+ x)
      elif stat['p-val'].values[0] > 0.99:
        table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == constraint), table_col].apply(lambda x: '\isWorse '+ x)

## RQ1

In [21]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_baseline'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_baseline', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)
baseline_df['statistical_parity'] = baseline_df['statistical_parity'].abs()
baseline_df['equal_opportunity'] = baseline_df['equal_opportunity'].abs()
baseline_df['average_odds'] = baseline_df['average_odds'].abs()

In [22]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [23]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_2796/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [24]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_new'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_new', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)
df_geg2['statistical_parity'] = df_geg2['statistical_parity'].abs()
df_geg2['equal_opportunity'] = df_geg2['equal_opportunity'].abs()
df_geg2['average_odds'] = df_geg2['average_odds'].abs()

In [25]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_2796/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [26]:
grouped_baseline['constraint'] = 'baseline'

In [27]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [28]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [29]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [30]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_2796/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [31]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.606$\pm$0.033,0.583$\pm$0.037,0.56$\pm$0.032,0.553$\pm$0.033,0.117$\pm$0.061,0.185$\pm$0.117,0.124$\pm$0.06
7,cmc,cp,0.606$\pm$0.03,0.576$\pm$0.038,0.561$\pm$0.032,0.556$\pm$0.034,0.067$\pm$0.051,0.145$\pm$0.081,0.076$\pm$0.056
8,cmc,dp,0.602$\pm$0.028,0.575$\pm$0.032,0.556$\pm$0.026,0.549$\pm$0.027,0.047$\pm$0.033,0.117$\pm$0.093,0.051$\pm$0.038
9,cmc,eo,0.601$\pm$0.034,0.574$\pm$0.038,0.56$\pm$0.031,0.557$\pm$0.032,0.038$\pm$0.035,0.135$\pm$0.095,0.061$\pm$0.042
1,crime,baseline,0.474$\pm$0.037,0.434$\pm$0.046,0.452$\pm$0.032,0.427$\pm$0.037,0.382$\pm$0.062,0.313$\pm$0.23,0.252$\pm$0.137
10,crime,cp,0.355$\pm$0.031,0.373$\pm$0.031,0.333$\pm$0.029,0.315$\pm$0.033,0.093$\pm$0.053,0.207$\pm$0.114,0.104$\pm$0.052
11,crime,dp,0.406$\pm$0.04,0.403$\pm$0.035,0.397$\pm$0.038,0.392$\pm$0.035,0.046$\pm$0.028,0.153$\pm$0.088,0.08$\pm$0.061
12,crime,eo,0.333$\pm$0.077,0.41$\pm$0.107,0.307$\pm$0.081,0.246$\pm$0.11,0.128$\pm$0.111,0.129$\pm$0.206,0.088$\pm$0.121
2,drug,baseline,0.687$\pm$0.023,0.618$\pm$0.033,0.614$\pm$0.018,0.611$\pm$0.025,0.213$\pm$0.125,0.285$\pm$0.164,0.177$\pm$0.1


In [32]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park
cmc
crime
law
obesity
wine
drug
park


In [33]:
table_tex.to_latex('results_summary.tex', index=False, escape=False)

### Comparison with DEMV

In [28]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_demv'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_demv', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [29]:
grouped_demv = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()
grouped_demv = grouped_demv.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_41761/3553521475.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_demv = grouped_demv.drop(columns='fold')


In [30]:
grouped_demv['constraint'] = 'demv'

In [31]:
full_data = pd.concat([grouped_demv, grouped_geg], ignore_index=True)

In [32]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)
full_data.sort_values(by=['data'], inplace=True)
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_41761/1697430065.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [33]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

obesity
crime
cmc
wine
law
drug
park
obesity
crime
cmc
wine
law
drug
park
obesity
crime
cmc
wine
law
drug
park
obesity
crime
cmc
wine
law
drug
park
obesity
crime
cmc
wine
law
drug
park
obesity
crime
cmc
wine
law
drug
park


In [34]:
table_tex.to_latex('results_demv_comparison.tex', index=False, escape=False)

## RQ2 

In [3]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_baseline_rq2'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_baseline_rq2', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [4]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [5]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_65935/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [6]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq2'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq2', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [7]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_65935/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [8]:
grouped_baseline['constraint'] = 'baseline'

In [9]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [10]:
full_data.head()

data  accuracy           precision              recall            \
               mean       std      mean       std      mean       std   
0   adult  0.826665  0.006915  0.771957  0.012435  0.726586  0.009579   
1  compas  0.675209  0.021380  0.675343  0.019898  0.666093  0.019814   
2  german  0.745000  0.044284  0.692620  0.063216  0.661321  0.057729   
3   adult  0.769134  0.004880  0.738693  0.024915  0.535491  0.007229   
4   adult  0.824596  0.005983  0.774139  0.012452  0.710732  0.007729   

   f1_score           statistical_parity           equal_opportunity  \
       mean       std               mean       std              mean   
0  0.743824  0.009620          -0.179410  0.015724         -0.154284   
1  0.665794  0.021166          -0.173967  0.040158         -0.102123   
2  0.668692  0.059542          -0.209581  0.107955         -0.182181   
3  0.506410  0.012765          -0.008359  0.007381          0.012244   
4  0.732296  0.008604          -0.076991  0.015889          0.129999   

            average_odds           constraint  
        std         mean       std             
0  0.047611    -0.119063  0.025816   baseline  
1  0.030810    -0.149917  0.041596   baseline  
2  0.170483    -0.169542  0.129837   baseline  
3  0.030731     0.008721  0.016611         cp  
4  0.070503     0.058032  0.036233         dp

In [11]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [12]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_65935/93204417.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [13]:
compute_statistics('accuracy', 'acc', 'less')
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

NameError: name 'compute_statistics' is not defined

In [84]:
table_tex.to_latex('results_summary_rq2.tex', index=False, escape=False)

### Comparison with EG

In [24]:
def compute_statistics_eg(col_name, table_col, condition):
  for data in baseline_df['data'].unique():
    baseline_acc = baseline_df[(baseline_df['data'] == data) & (baseline_df['constraint'] == 'eg_dp')][col_name]
    
    geg_acc = df_geg2[(df_geg2['data'] == data) & (df_geg2['constraint'] == 'cp')][col_name]
    stat = wilcoxon(baseline_acc.abs(), geg_acc.abs(), alternative=condition)
    
    print(f"{data} - cp - p-value: {stat['p-val'].values[0]}")
    
        # if stat['p-val'].values[0] <= 0.05:
    #   table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == 'cp'), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == 'cp'), table_col].apply(lambda x: '\isBetter '+ x)
    # elif stat['p-val'].values[0] > 0.99:
    #   table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == 'cp'), table_col] = table_tex.loc[(table_tex['data'] == data) & (table_tex['constraint'] == 'cp'), table_col].apply(lambda x: '\isWorse '+ x)

In [15]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_eg'):
    if file.endswith('.csv'):
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_eg', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [16]:
baseline_df = baseline_df[baseline_df['constraint'] == 'eg_dp']

In [17]:
baseline_df

,fold,accuracy,precision,recall,f1_score,statistical_parity,equal_opportunity,average_odds,constraint,data
0,1,0.820000,0.791643,0.750850,0.766112,0.028509,0.028409,0.073728,eg_dp,german
2,3,0.740000,0.687500,0.642857,0.653333,-0.042523,0.037356,-0.101322,eg_dp,german
4,5,0.710000,0.659132,0.621324,0.627776,0.084991,0.049442,0.151994,eg_dp,german
6,7,0.770000,0.707995,0.655758,0.670251,-0.157738,-0.017361,-0.119395,eg_dp,german
8,9,0.720000,0.650480,0.629432,0.636175,-0.150000,-0.092838,-0.150315,eg_dp,german
10,11,0.710000,0.700000,0.659168,0.664002,0.058824,0.142857,0.146141,eg_dp,german
12,13,0.680000,0.548278,0.548278,0.548278,-0.218994,-0.519608,-0.122304,eg_dp,german
14,15,0.830000,0.839964,0.759191,0.781800,0.188347,0.055556,0.206349,eg_dp,german
16,17,0.780000,0.704687,0.693333,0.698465,0.077236,0.038462,0.217760,eg_dp,german
18,19,0.660000,0.611519,0.586813,0.587178,-0.081140,0.081818,-0.030519,eg_dp,german


In [18]:
grouped_demv = baseline_df.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index()
grouped_demv = grouped_demv.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_65935/2876153373.py:2: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_demv = grouped_demv.drop(columns='fold')


In [19]:
full_data = pd.concat([grouped_demv, grouped_geg[grouped_geg['constraint'] == 'cp']], ignore_index=True)

In [20]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)
full_data.sort_values(by=['data'], inplace=True)
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_65935/1697430065.py:9: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [21]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,adult,eg_dp,0.817$\pm$0.006,0.761$\pm$0.014,0.697$\pm$0.009,0.718$\pm$0.01,-0.015$\pm$0.018,0.269$\pm$0.069,0.15$\pm$0.037
3,adult,cp,0.769$\pm$0.005,0.739$\pm$0.025,0.535$\pm$0.007,0.506$\pm$0.013,-0.008$\pm$0.007,0.012$\pm$0.031,0.009$\pm$0.017
1,compas,eg_dp,0.665$\pm$0.019,0.665$\pm$0.021,0.654$\pm$0.019,0.654$\pm$0.019,-0.013$\pm$0.05,0.061$\pm$0.047,0.009$\pm$0.048
4,compas,cp,0.611$\pm$0.031,0.695$\pm$0.028,0.578$\pm$0.025,0.52$\pm$0.049,0.001$\pm$0.072,0.014$\pm$0.051,0.019$\pm$0.073
2,german,eg_dp,0.742$\pm$0.057,0.69$\pm$0.083,0.655$\pm$0.066,0.663$\pm$0.072,-0.021$\pm$0.13,-0.02$\pm$0.186,0.027$\pm$0.147
5,german,cp,0.721$\pm$0.048,0.747$\pm$0.118,0.549$\pm$0.039,0.51$\pm$0.073,-0.026$\pm$0.067,-0.037$\pm$0.052,-0.002$\pm$0.068


In [25]:
compute_statistics_eg('accuracy', 'acc', 'less')
compute_statistics_eg('precision', 'prec', 'less')
compute_statistics_eg('recall', 'rec', 'less')
compute_statistics_eg('f1_score', 'f1', 'less')
compute_statistics_eg('statistical_parity', 'spd', 'greater')
compute_statistics_eg('equal_opportunity', 'eod', 'greater')
compute_statistics_eg('average_odds', 'aod', 'greater')

german - cp - p-value: 0.8466796875
adult - cp - p-value: 1.0
compas - cp - p-value: 1.0
german - cp - p-value: 0.052734375
adult - cp - p-value: 0.9931640625
compas - cp - p-value: 0.0244140625
german - cp - p-value: 1.0
adult - cp - p-value: 1.0
compas - cp - p-value: 1.0
german - cp - p-value: 1.0
adult - cp - p-value: 1.0
compas - cp - p-value: 1.0
german - cp - p-value: 0.0419921875
adult - cp - p-value: 0.0244140625
compas - cp - p-value: 0.8623046875
german - cp - p-value: 0.1611328125
adult - cp - p-value: 0.0009765625
compas - cp - p-value: 0.1162109375
german - cp - p-value: 0.0068359375
adult - cp - p-value: 0.0009765625
compas - cp - p-value: 0.8623046875


In [23]:
table_tex.to_latex('results_eg_comparison.tex', index=False, escape=False)

## RQ4

In [126]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_models'):
    if file.endswith('.csv') and '_rf' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_models', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [127]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [128]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_41761/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [129]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq4'):
    if file.endswith('.csv') and '_rf' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq4', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [130]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_41761/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [131]:
grouped_baseline['constraint'] = 'baseline'

In [132]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [133]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [134]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [135]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_41761/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [136]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.98$\pm$0.015,0.979$\pm$0.016,0.977$\pm$0.017,0.978$\pm$0.017,0.136$\pm$0.08,0.022$\pm$0.087,0.012$\pm$0.045
7,cmc,cp,0.794$\pm$0.041,0.833$\pm$0.022,0.833$\pm$0.032,0.79$\pm$0.043,0.04$\pm$0.16,0.01$\pm$0.062,-0.036$\pm$0.083
8,cmc,dp,0.868$\pm$0.022,0.869$\pm$0.02,0.882$\pm$0.02,0.86$\pm$0.025,0.029$\pm$0.092,0.015$\pm$0.083,-0.058$\pm$0.068
9,cmc,eo,0.984$\pm$0.009,0.985$\pm$0.008,0.981$\pm$0.01,0.983$\pm$0.009,0.141$\pm$0.084,0.034$\pm$0.078,0.019$\pm$0.04
1,crime,baseline,0.499$\pm$0.031,0.471$\pm$0.03,0.478$\pm$0.025,0.466$\pm$0.027,0.413$\pm$0.069,0.483$\pm$0.232,0.355$\pm$0.14
10,crime,cp,0.415$\pm$0.04,0.442$\pm$0.032,0.386$\pm$0.03,0.364$\pm$0.036,0.19$\pm$0.046,0.379$\pm$0.226,0.197$\pm$0.113
11,crime,dp,0.437$\pm$0.04,0.441$\pm$0.033,0.411$\pm$0.022,0.395$\pm$0.03,0.155$\pm$0.055,0.307$\pm$0.244,0.139$\pm$0.124
12,crime,eo,0.507$\pm$0.036,0.473$\pm$0.036,0.484$\pm$0.029,0.47$\pm$0.034,0.424$\pm$0.061,0.497$\pm$0.194,0.367$\pm$0.117
2,drug,baseline,0.676$\pm$0.032,0.606$\pm$0.033,0.597$\pm$0.027,0.597$\pm$0.028,0.143$\pm$0.125,0.177$\pm$0.194,0.108$\pm$0.128


In [137]:
compute_statistics('accuracy', 'acc', 'less')
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime
law
obesity
park
drug
cmc
wine
crime


In [138]:
table_tex.to_latex('results_summary_rq4_rf.tex', index=False, escape=False)

## SVM

In [54]:
baseline_df = pd.DataFrame()
for file in os.listdir('results_models'):
    if file.endswith('.csv') and '_svm' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_models', file))
        df['data'] = data
        baseline_df = pd.concat([baseline_df, df], ignore_index=True)

In [55]:
grouped = baseline_df.groupby('data').agg(['mean', 'std']).reset_index()

In [56]:
grouped_baseline = grouped.drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/2910365511.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_baseline = grouped.drop(columns='fold')


In [57]:
df_geg2 = pd.DataFrame()
for file in os.listdir('results_geg_rq4'):
    if file.endswith('.csv') and '_svm' in file:
        data = file.split('_')[0]
        df = pd.read_csv(os.path.join('results_geg_rq4', file))
        df['data'] = data
        df_geg2 = pd.concat([df_geg2, df], ignore_index=True)

In [58]:
grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/157383463.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  grouped_geg = df_geg2.groupby(['data', 'constraint']).agg(['mean', 'std']).reset_index().drop(columns='fold')


In [59]:
grouped_baseline['constraint'] = 'baseline'

In [60]:
full_data = pd.concat([grouped_baseline.assign(constraint='baseline'), grouped_geg], ignore_index=True)

In [61]:
full_data['acc'] = full_data[('accuracy', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('accuracy', 'std')].round(3).astype(str)
full_data['prec'] = full_data[('precision', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('precision', 'std')].round(3).astype(str)
full_data['rec'] = full_data[('recall', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('recall', 'std')].round(3).astype(str)
full_data['f1'] = full_data[('f1_score', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('f1_score', 'std')].round(3).astype(str)
full_data['spd'] = full_data[('statistical_parity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('statistical_parity', 'std')].round(3).astype(str)
full_data['eod'] = full_data[('equal_opportunity', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('equal_opportunity', 'std')].round(3).astype(str)
full_data['aod'] = full_data[('average_odds', 'mean')].round(3).astype(str) + '$\pm$' + full_data[('average_odds', 'std')].round(3).astype(str)

In [62]:
full_data.sort_values(by=['data', 'constraint'], inplace=True)

In [63]:
table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)

/var/folders/4c/mgvn0dc97_9gst9l7jbv9n640000gn/T/ipykernel_4166/1130035862.py:1: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  table_tex = full_data.drop(columns=['accuracy', 'precision', 'recall', 'f1_score', 'statistical_parity', 'equal_opportunity', 'average_odds'], axis=1)


In [64]:
table_tex

,data,constraint,acc,prec,rec,f1,spd,eod,aod
,,,,,,,,,
0,cmc,baseline,0.777$\pm$0.039,0.835$\pm$0.028,0.777$\pm$0.029,0.773$\pm$0.036,0.076$\pm$0.089,0.031$\pm$0.242,0.008$\pm$0.12
7,cmc,cp,0.646$\pm$0.072,0.696$\pm$0.051,0.672$\pm$0.056,0.64$\pm$0.068,0.036$\pm$0.121,0.005$\pm$0.197,-0.02$\pm$0.128
8,cmc,dp,0.273$\pm$0.039,0.61$\pm$0.109,0.372$\pm$0.014,0.211$\pm$0.029,0.027$\pm$0.046,0.02$\pm$0.047,0.022$\pm$0.038
9,cmc,eo,0.484$\pm$0.187,0.665$\pm$0.081,0.542$\pm$0.146,0.468$\pm$0.2,0.052$\pm$0.09,0.026$\pm$0.088,0.017$\pm$0.077
1,crime,baseline,0.234$\pm$0.042,0.063$\pm$0.03,0.198$\pm$0.011,0.086$\pm$0.025,0.001$\pm$0.014,-0.016$\pm$0.063,-0.006$\pm$0.032
10,crime,cp,0.233$\pm$0.038,0.067$\pm$0.036,0.199$\pm$0.008,0.086$\pm$0.024,-0.002$\pm$0.032,-0.012$\pm$0.054,-0.007$\pm$0.037
11,crime,dp,0.235$\pm$0.044,0.065$\pm$0.031,0.199$\pm$0.011,0.087$\pm$0.025,0.006$\pm$0.016,-0.014$\pm$0.062,-0.004$\pm$0.029
12,crime,eo,0.235$\pm$0.034,0.073$\pm$0.059,0.201$\pm$0.004,0.086$\pm$0.03,-0.005$\pm$0.017,-0.02$\pm$0.057,-0.011$\pm$0.031
2,drug,baseline,0.501$\pm$0.034,0.466$\pm$0.049,0.466$\pm$0.047,0.441$\pm$0.045,0.045$\pm$0.111,-0.016$\pm$0.304,0.001$\pm$0.153


In [65]:
compute_statistics('precision', 'prec', 'less')
compute_statistics('recall', 'rec', 'less')
compute_statistics('f1_score', 'f1', 'less')
compute_statistics('statistical_parity', 'spd', 'greater')
compute_statistics('equal_opportunity', 'eod', 'greater')
compute_statistics('average_odds', 'aod', 'greater')

cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity
drug
crime
cmc
law
park
wine
obesity


/Users/giord/miniconda3/envs/ds/lib/python3.10/site-packages/scipy/stats/_wilcoxon.py:172: RuntimeWarning: invalid value encountered in scalar divide
  z = (r_plus - mn) / se
/Users/giord/miniconda3/envs/ds/lib/python3.10/site-packages/pingouin/nonparametric.py:493: RuntimeWarning: invalid value encountered in scalar divide
  rbc = r_plus / rsum - r_minus / rsum


drug
crime
cmc
law
park
wine
obesity
drug
crime


In [66]:
table_tex.to_latex('results_summary_rq4_svm.tex', index=False, escape=False)